머신 러닝의 목표는 처음 본 데이터에서 잘 작동하는 **일반화**된 모델을 얻는 것입니다. 여기에서 **과대적합**은 중요 장애물입니다. **과대적합**되기 시작하면 훈련 데이터의 성능에 비해 처음 본 데이터에 대한 성능이 좋아지지 않습니다.

### 훈련, 검증, 테스트 세트

모델 평가의 핵심은 가용한 데이터를 항상 **훈련**, **검증**, **테스트** 3개의 세트로 나누는 것입니다. 훈련 세트에서 모델을 훈련하고, 검증 세트에서 모델을 평가합니다.테스트 세트에서 최정적으로 딱 한 번 모델을 테스트합니다.

모델을 개발할 때 항상 모델의 설정을 튜닝합니다. 층의 수나 유닛 수를 튜닝합니다. 이런 파라미터를 네트워크의 가중치와 구분하기 위해 **하이퍼파라미터 (hyperparameter)** 라고 부릅니다. 검증 세트에서 모델의 성능을 평가하여 이런 튜닝을 수행합니다. 본질적으로 이런 튜닝도 어떤 파라미터 공간에서 좋은 설정을 찾는 **학습**입니다. 검증 세트의 성능을 기반으로 모델의 설정을 튜닝하면 검증 세트로 모델을 직접 훈련하지 않더라도 빠르게 검증 세트에 **과대적합**될 수 있습니다.

이런 현상의 핵심은 **정보 누설 (information leak)**에 있습니다. 검증 세트의 모델 성능에 기반하여 모델의 하이퍼파라미터를 조정할 때마다 정증 데이터에 관한 정보가 모델로 새게 됩니다. 검증 세트에 평가한 결과를 가지고 다시 모델을 조정하는 과정을 여러번 반복하다보면 결국 검증 데이터에 맞추어 최적화된 모델이 만들어지고 검증 데이터에 의도적으로 잘 수행되는 모델이 만들어집니다. 튜닝된 모델은 일반화 성능을 왜곡시킵니다.

#### 단순 홀드아웃 검증

데이터의 일정량을 테스트 세트로 떼어 놓습니다. 남은 데이터에서 훈련하고 테스트 세트로 평가 받습니다. 정보 누설을 막기 위해 테스트 세트를 사용하여 모델을 튜닝해서는 안됩니다. 이런 이유로 검증 세트도 따로 떼어 놓아야 합니다.

![](../images/4-1-holdout.png)

##### 홀드아웃 검증 구현 예시

<br>
</br>

```python
num_validation_samples = 10000

np.random.shuffle(data) ## 데이터를 섞는 것이 일반적으로 좋습니다

validation_data = data[:num_validation_samples] ## 검증 세트를 만듭니다
data = data[num_validation_samples:]

training_data == data[:] ## 훈련 세트를 만듭니다

## 훈련 세트에서 모델을 훈련하고 검증 세트로 평가합니다.
model = get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data)

## 여기서 모델을 튜닝하고, 다시 훈련하고, 평가하고, 다시 튜닝하고.. 반복

## 하이퍼파라미터 튜닝이 끝나면 테스트 데이터를 제외한 모든 데이터를 사용하여 모델을 다시 훈련시킵니다.
model = get_model()
model.train(np.concatenate([training_data, validation_data]))
test_score = model.evaluate(test_data)
```

이 방법은 너무 단순해서 한가지 단점이 있습니다. 데이터가 적을때는 검증 세트와 테스트 세트의 샘플이 너무 적어 주어진 전체 데이터를 통계적으로 대표하지 못할 수 있습니다. 이를 해결하기 위하여 K-겹 교차 검증을 이용합니다.

#### K-겹 교차 검증

이 방식에는 데이터를 동일한 크기를 가진 K개 분할로 나눕니다. 각 분할 i에 대해 남은 K - 1개의 분할로 무델을 훈련하고 분할 i 에서 모델을 평가합니다. 최종 점수는 이렇게 얻은 K개의 점수를 평균합니다.

<br></br>
![](../images/4-1-Kfold.png)

##### K-겹 교차 검증 구현 예

<br></br>
```python
k = 4
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_scores = []

for fold in range(k):
    validation_data = data[num_validation_samples * fold: 
                           num_validation_samples * (fold + 1)]
    training_data = data[:num_validation_samples * fold] + 
                    data[num_validation_samples * (fold + 1):]
    
    
    model = get_model()
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)
    
    validation_score = np.average(validation_scores)
    
    model = get_model()
    model.train(data)
    test_score = model.evaluate(test_data)
```

#### 셔플링을 사용한 반복 K-겹 교차 검증 (Iterated K-fold Cross-validation)

이 방법은 K-겹 교차 검증을 여러 번 적용하되 K개의 분할로 나누기 전에 매번 데이터를 무작위로 섞습니다. 최종 점수는 모든 K-겹 교차 검증을 실행에서 얻는 점수의 평균이 됩니다. 결국 P * K개 (P는 반복 횟수)의 모델을 훈련하고 평가하므로 비용이 매우 큽니다.